In [1]:
# sys imports
import os
import sys
import warnings
import logging
import nest_asyncio

# data handling
import pandas as pd

# RAG
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    Response
)
from llama_index.core.evaluation import (
    BatchEvalRunner,
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    CorrectnessEvaluator,
    RetrieverEvaluator,
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
    DatasetGenerator
)
from llama_index.llms.openai import OpenAI
from llama_index.llms.huggingface import HuggingFaceLLM

nest_asyncio.apply()
sys.path.append(os.path.join('/Users/juandiegogallegoquiceno/Desktop/pinacle/secrets'))

# import secrets
from hf_secrets import api_token as hf_token
from openai_secrets import api_key as oai_token

/opt/homebrew/Caskroom/miniforge/base/envs/llamaindex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/llamaindex/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
# assign as environ vars
os.environ['OPENAI_API_KEY'] = oai_token
os.environ['HF_TOKEN'] = hf_token

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
HF_TOKEN = os.environ['HF_TOKEN']

## Read data

In [3]:
reader = SimpleDirectoryReader('../data')
documents = reader.load_data('Final Policy document_LICs New Jeevan Shanti_V05_logo.pdf')
len(documents)

Loading files: 100%|██████████| 1/1 [00:00<00:00,  1.32file/s]


21

## Embedding types

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [5]:
# load embed model
hf_embed_model = HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5')

/opt/homebrew/Caskroom/miniforge/base/envs/llamaindex/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## LLMs

In [6]:
gpt3_5 = OpenAI(model='gpt-3.5-turbo', temperature=0.1)
llama2 = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-hf",
    generate_kwargs={"temperature": 0.1, "do_sample": False}
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.01s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### With GPT 3.5 Turbo

In [7]:
# generate some data from document
num_eval_nodes = 10
data_generator = DatasetGenerator.from_documents(documents, llm=gpt3_5)
eval_dataset = data_generator.generate_dataset_from_nodes(num=num_eval_nodes)

/opt/homebrew/Caskroom/miniforge/base/envs/llamaindex/lib/python3.9/site-packages/llama_index/core/evaluation/dataset_generation.py:213: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(
/opt/homebrew/Caskroom/miniforge/base/envs/llamaindex/lib/python3.9/site-packages/llama_index/core/evaluation/dataset_generation.py:310: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [8]:
# now get eval questions and eval answers
eval_questions = [ex[0] for ex in eval_dataset.qr_pairs]
eval_answers = [ex[1] for ex in eval_dataset.qr_pairs]

print(len(eval_questions))
print(eval_questions[0], eval_answers[0])

10
What is the registration number of LIC's New Jeevan Shanti policy? The registration number of LIC's New Jeevan Shanti policy is 512.


In [9]:
# get and eval query example
eval_query = eval_questions[0]

#### Vector store

In [10]:
# with Hugginface Embedding BAAI/bge-small-en-v1.5
index = VectorStoreIndex.from_documents(documents, embed_model=hf_embed_model, llm=gpt3_5)
# query engine to generate response
query_engine = index.as_query_engine()
# define retriever as well
retriever = index.as_retriever(similarity_top_k=3)
# get nodes
nodes = retriever.retrieve(eval_query)

#### Evaluation

In [11]:
# util function to get metric result from batch eval runner
def get_eval_results(key, eval_results):
    results = eval_results[f'{key}']
    correct = 0
    for result in results:
        if result.passing:
            correct += 1
    score = correct / len(results)
    print(f"{key} Score: {score}")
    return score

In [13]:
# define a runner evaluation instance
runner = BatchEvalRunner(
    {
        "faithfulness": FaithfulnessEvaluator(llm=gpt3_5),
        "relevancy": RelevancyEvaluator(llm=gpt3_5),
        "correctness": CorrectnessEvaluator(llm=gpt3_5)
    },
    workers=8
)
# evaluate
eval_results = await runner.aevaluate_queries(
    query_engine,
    queries=eval_questions,
    reference=eval_answers
)

In [15]:
# see results
for key in ["faithfulness", "relevancy", "correctness"]:
    get_eval_results(key, eval_results)

faithfulness Score: 0.8
relevancy Score: 0.6
correctness Score: 0.4


Seems that LLM is not hallucinating but answers has not an optimum quality based on correctnes score and are not the most relevant for the passed query.

### With Llama2

#### Vector store

In [16]:
# with Hugginface Embedding BAAI/bge-small-en-v1.5
index = VectorStoreIndex.from_documents(documents, embed_model=hf_embed_model, llm=llama2)
# query engine to generate response
query_engine = index.as_query_engine()
# define retriever as well
retriever = index.as_retriever(similarity_top_k=3)
# get nodes
nodes = retriever.retrieve(eval_query)

#### Evaluation

In [17]:
# define a runner evaluation instance
runner = BatchEvalRunner(
    {
        "faithfulness": FaithfulnessEvaluator(llm=llama2),
        "relevancy": RelevancyEvaluator(llm=llama2),
        "correctness": CorrectnessEvaluator(llm=llama2)
    },
    workers=8
)
# evaluate
eval_results = await runner.aevaluate_queries(
    query_engine,
    queries=eval_questions,
    reference=eval_answers
)

/opt/homebrew/Caskroom/miniforge/base/envs/llamaindex/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/llamaindex/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
# see results
for key in ["faithfulness", "relevancy", "correctness"]:
    get_eval_results(key, eval_results)

faithfulness Score: 0.8
relevancy Score: 0.6
correctness Score: 0.4
